# ***Controller*** notes (controller.py)

## *init()*

```python
for i in range(xsize):
    for j in range(ysize):
        if random.random() < prob:
            new_cell = HealthyCell(random.randint(0, 4))
            self.grid.cells[i, j].append(new_cell)
```
L'obiettivo generale di questo codice è inizializzare la griglia dell'ambiente cellulare. Le cellule sane vengono distribuite casualmente sulla griglia, con una probabilità determinata dal parametro hcells. Successivamente, viene posizionata una cellula cancerosa al centro della griglia.

## *observeSegmentation()* e *observeDensity()*

Consideriamo solo observeSegmentation() poichè per *observeDensity()* vale la stessa cosa.
```python
def observeSegmentation(self):
    """Produce observation of type segmentation"""
    seg = np.vectorize(lambda x:x.pixel_type())
    return seg(self.grid.cells)
```

* *lambda x: x.pixel_type()*: Crea una funzione anonima (*lambda*) che prende un parametro *x* e restituisce il risultato di *x.pixel_type()*
* *np.vectorize()*: Accetta un'operazione scalare (cioè che opera su un singolo elemento) e la trasforma in un'operazione che può essere applicata elemento per elemento su un array. *lambda* viene usata per definire l'operazione scalare che verrà applicata su ogni cellula della griglia.
* *return seg(self.grid.cells)*: Il risultato è un array dove ogni elemento rappresenta il tipo di una cellula (ad esempio, potrebbe essere "cellula sana" o "cellula cancerosa"). Questo array viene restituito come l'osservazione "segmentata" dell'ambiente.

## go()
La funzione *go()* esegue un ciclo di simulazione per un certo numero di passi temporali.

* Basandonci sul fatto che nella funzione ogni ciclo (passo) corrisponde a un incremento del contatore del tempo (*self.tick += 1*) e che ogni 24 passi si esegue l'operazione *self.grid.compute_center()*, sembra ragionevole assumere che ogni passo della simulazione rappresenti un'ora (steps = 1 corrisponde a 1 ora).

*   ```python
    self.tick += 1
    ```
    Aggiorna il contatore del tempo (tick) ad ogni passo della simulazione.

*   ```python
    if self.draw_step > 0 and self.tick % self.draw_step == 0:
    self.update_plots()
    ```
    Se il parametro self.draw_step è maggiore di 0 e l'indice temporale (*tick*) è un multiplo di *self.draw_step*, viene chiamata la funzione *self.update_plots()* per aggiornare le visualizzazioni grafiche della simulazione.

## plot_init()

*   ```python
    matplotlib.use("TkAgg")
    ```
    Imposta il backend di Matplotlib su TkAgg, il che significa che i grafici verranno visualizzati in finestre separate (spesso usato per visualizzazioni interattive). Il *backend* è responsabile di rendere visibili i grafici e le figure su diverse piattaforme e dispositivi. Ogni backend gestisce come il grafico viene disegnato e mostrato. Esistono due tipi principali di backend:
    * Backend interattivi (GUI) come *TkAgg*
    * Backend non interattivi come *pdf*
    Quando si imposta matplotlib.use("TkAgg"), si dice a Matplotlib di utilizzare Tkinter per creare una finestra interattiva che mostrerà i grafici. Agg gestisce il disegno vero e proprio del grafico, fornendo un rendering di alta qualità, mentre Tkinter gestisce la finestra e l'interfaccia utente.

*   ```python
    plt.ion()
    ```
    Abilita la modalità interattiva di Matplotlib, che consente di aggiornare i grafici in tempo reale senza bloccare l'esecuzione del codice.

*   ```python
    self.fig, axs = plt.subplots(2,2, constrained_layout=True)
    ```
    Crea una figura con una griglia di sottotrame (2x2), ottimizzando automaticamente il layout con *constrained_layout=True* per evitare sovrapposizioni tra i componenti del grafico.

*   ```python
    self.cell_plot.imshow(
                    [[patch_type_color(self.grid.cells[i][j]) for j in range(self.grid.ysize)] for i in range(self.grid.xsize)])
    ```
    *  *imshow()*:  Visualizza una rappresentazione grafica dei tipi di cellule all'interno della griglia della simulazione. L'elemento tra le parentesi è una matrice in moodo da ottenere, alla fine, una matrice bidimensionale, dove ogni elemento è il risultato della funzione *patch_type_color()*
    * *patch_type_color()*: Prende in input un elemento della griglia e restituisce il colore corrispondente, che viene poi utilizzato per visualizzare la cellula nel grafico.
        *   ```python
            def patch_type_color(patch):
                if len(patch) == 0:
                    return 0, 0, 0
                else:
                    return patch[0].cell_color()
            ``` 
        * *patch*: Rappresenta un gruppo di cellule che occupano una singola posizione della griglia.
        * La funzione controlla prima se il "patch" è vuoto, cioè se la lista di cellule in quella posizione della griglia è vuota. In questo caso restituisce il colore nero (0, 0, 0).
        * Se il "patch" non è vuoto, la funzione restituisce il colore della prima cellula nel "patch".
        * *patch[0]*: Rappresenta la prima cellula nella lista, e la funzione *cell_color()* viene chiamata su quella cellula per ottenere il colore associato a quel tipo di cellula.
        * Il colore di ogni tipo di cellula è impostato nella funzione *cell_color()* presente all'interno di ciascuna classe di cellule nel file *cell.py*.

*   ```python
        self.cell_density_plot.imshow(
            [[len(self.grid.cells[i][j]) for j in range(self.grid.ysize)] for i in range(self.grid.xsize)])
    ```
    * Similmente alla visualizzazione dei tipi di cellule, qui viene visualizzata la densità delle cellule in ogni punto della griglia.
    * All'interno delle parentesi di *imshow* si ha una matrice in cui gli elementi rappresentano il numero di cellule all'interno dell'elemento *self.grid.cells[i][j]*.
    * Il colore cambia in automatico in base alla densità do cellule (mappa di calore).

## update_plots()

Questa funzione è uguale a *plot_init()* con l'unica differenza nella mancanza della dichiarazione iniziale dei grafici. Serve per aggiornare tutti e 4 io grafici presenti nel subplot.